## Introduction

This tutorial demonstrates Open3D tensor implementation.

Like tensorflow and pytorch, Open3D provides its own implementation of a tensor, a multidimensional and homogeneous matrix containing elements of single data type. It supports various basis mathematical operators, as well as GPU operations.

In [1]:
import open3d as o3d
import open3d.core as o3c
import numpy as np

## 1. Tensor creation

We can create an Open3D tensor in similar way with pytorch or tensorflow.
A tensor can be initialize from a list or numpy array, with data type and device arguments.

In [2]:
# Tensor from list.
a = o3c.Tensor([0, 1, 2])
print("Created from list:\n{}".format(a))

# Tensor from Numpy.
a = o3c.Tensor(np.array([0, 1, 2]))
print("\nCreated from numpy array:\n{}".format(a))

# Dtype and inferred from list.
a_float = o3c.Tensor([0.0, 1.0, 2.0])
print("\nDefault dtype and device:\n{}".format(a_float))

# Specify dtype.
a = o3c.Tensor(np.array([0, 1, 2]), dtype=o3c.Dtype.Float64)
print("\nSpecified data type:\n{}".format(a))

# Specify device.
a = o3c.Tensor(np.array([0, 1, 2]), device=o3c.Device("CUDA:0"))
print("\nSpecified device:\n{}".format(a))

Created from list:
[0 1 2]
Tensor[shape={3}, stride={1}, Int64, CPU:0, 0x562268c62ba0]

Created from numpy array:
[0 1 2]
Tensor[shape={3}, stride={1}, Int64, CPU:0, 0x562268c94080]

Default dtype and device:
[0.0 1.0 2.0]
Tensor[shape={3}, stride={1}, Float64, CPU:0, 0x562268c442d0]

Specified data type:
[0.0 1.0 2.0]
Tensor[shape={3}, stride={1}, Float64, CPU:0, 0x5622690080c0]

Specified device:
[0 1 2]
Tensor[shape={3}, stride={1}, Int64, CUDA:0, 0x7fd2f9600000]


## 2. Tensor properties

Like pytorch, the properties of an Open3D tensor can be accessed like this.

In [3]:
vals = np.array((range(24))).reshape(2, 3, 4)
a = o3c.Tensor(vals, dtype=o3c.Dtype.Float64, device=o3c.Device("CUDA:0"))
print(f"a.shape: {a.shape}")
print(f"a.strides: {a.strides}")
print(f"a.dtype: {a.dtype}")
print(f"a.device: {a.device}")
print(f"a.ndim: {a.ndim}")

a.shape: SizeVector[2, 3, 4]
a.strides: SizeVector[12, 4, 1]
a.dtype: Float64
a.device: CUDA:0
a.ndim: 3


## 3. Tensor Copy & Device transfer
A tensor can be transferred between devices.

In [4]:
# Host -> Device.
a_cpu = o3c.Tensor([0, 1, 2])
a_gpu = a_cpu.cuda(0)
print(a_gpu)

# Device -> Host.
a_gpu = o3c.Tensor([0, 1, 2], device=o3c.Device("CUDA:0"))
a_cpu = a_gpu.cpu()
print(a_cpu)

# Device -> another Device.
a_gpu_0 = o3c.Tensor([0, 1, 2], device=o3c.Device("CUDA:0"))
a_gpu_1 = a_gpu_0.cuda(0)
print(a_gpu_1)

[0 1 2]
Tensor[shape={3}, stride={1}, Int64, CUDA:0, 0x7fd2f9600000]
[0 1 2]
Tensor[shape={3}, stride={1}, Int64, CPU:0, 0x5622bd69f800]
[0 1 2]
Tensor[shape={3}, stride={1}, Int64, CUDA:0, 0x7fd2f9600000]


## 4. Data types.
The data type of a tensor can be casted.

In [5]:
# E.g. float -> int
a = o3c.Tensor([0.1, 1.5, 2.7])
b = a.to(o3c.Dtype.Int32)
print(a)
print(b)

[0.1 1.5 2.7]
Tensor[shape={3}, stride={1}, Float64, CPU:0, 0x5622a3d1a7b0]
[0 1 2]
Tensor[shape={3}, stride={1}, Int32, CPU:0, 0x56229e6f7030]


## 5. Conversion from pytorch's tensor

Open3D tensor can be smoothly converted to pytorch tensor using the below code.

In [6]:
import torch
import torch.utils.dlpack

# From PyTorch
th_a = torch.ones((5,)).cuda(0)
o3_a = o3c.Tensor.from_dlpack(torch.utils.dlpack.to_dlpack(th_a))
print(f"th_a: {th_a}")
print(f"o3_a: {o3_a}")
print("")

# Changes to PyTorch array reflects on open3d Tensor and vice versa
th_a[0] = 100
o3_a[1] = 200
print(f"th_a: {th_a}")
print(f"o3_a: {o3_a}")

# To PyTorch
o3_a = o3c.Tensor([1, 1, 1, 1, 1], device=o3c.Device("CUDA:0"))
th_a = torch.utils.dlpack.from_dlpack(o3_a.to_dlpack())
o3_a = o3c.Tensor.from_dlpack(torch.utils.dlpack.to_dlpack(th_a))
print(f"th_a: {th_a}")
print(f"o3_a: {o3_a}")
print("")

# Changes to PyTorch array reflects on open3d Tensor and vice versa
th_a[0] = 100
o3_a[1] = 200
print(f"th_a: {th_a}")
print(f"o3_a: {o3_a}")

th_a: tensor([1., 1., 1., 1., 1.], device='cuda:0')
o3_a: [1.0 1.0 1.0 1.0 1.0]
Tensor[shape={5}, stride={1}, Float32, CUDA:0, 0x7fd118200000]

th_a: tensor([100., 200.,   1.,   1.,   1.], device='cuda:0')
o3_a: [100.0 200.0 1.0 1.0 1.0]
Tensor[shape={5}, stride={1}, Float32, CUDA:0, 0x7fd118200000]
th_a: tensor([1, 1, 1, 1, 1], device='cuda:0')
o3_a: [1 1 1 1 1]
Tensor[shape={5}, stride={1}, Int64, CUDA:0, 0x7fd2f9600a00]

th_a: tensor([100, 200,   1,   1,   1], device='cuda:0')
o3_a: [100 200 1 1 1]
Tensor[shape={5}, stride={1}, Int64, CUDA:0, 0x7fd2f9600a00]


## 6. Binary element-wise operation
Various binary element-wise operations are supported.

In [7]:
a = o3c.Tensor([1, 1, 1], dtype=o3c.Dtype.Float32)
b = o3c.Tensor([2, 2, 2], dtype=o3c.Dtype.Float32)
print("a + b = {}".format(a + b))
print("a - b = {}".format(a - b))
print("a * b = {}".format(a * b))
print("a / b = {}".format(a / b))

a + b = [3.0 3.0 3.0]
Tensor[shape={3}, stride={1}, Float32, CPU:0, 0x5623516f0080]
a - b = [-1.0 -1.0 -1.0]
Tensor[shape={3}, stride={1}, Float32, CPU:0, 0x56235028cfd0]
a * b = [2.0 2.0 2.0]
Tensor[shape={3}, stride={1}, Float32, CPU:0, 0x5623516f0080]
a / b = [0.5 0.5 0.5]
Tensor[shape={3}, stride={1}, Float32, CPU:0, 0x56235028cfd0]


## 7. Unary element-wise operation
Various Unary element-wise operations are supported.

In [8]:
a = o3c.Tensor([4, 9, 16], dtype=o3c.Dtype.Float32)
print("a = {}\n".format(a))
print("a.sqrt = {}\n".format(a.sqrt()))
print("a.sin = {}\n".format(a.sin()))
print("a.cos = {}\n".format(a.cos()))

# Inplace operation
a.sqrt_()
print(a)

a = [4.0 9.0 16.0]
Tensor[shape={3}, stride={1}, Float32, CPU:0, 0x5623502e9100]

a.sqrt = [2.0 3.0 4.0]
Tensor[shape={3}, stride={1}, Float32, CPU:0, 0x5623502f1860]

a.sin = [-0.756802 0.412119 -0.287903]
Tensor[shape={3}, stride={1}, Float32, CPU:0, 0x56234fb4f140]

a.cos = [-0.653644 -0.91113 -0.95766]
Tensor[shape={3}, stride={1}, Float32, CPU:0, 0x562350155390]

[2.0 3.0 4.0]
Tensor[shape={3}, stride={1}, Float32, CPU:0, 0x5623502e9100]


## 8. Reduction
It supports various reduction operations, such as sum, mean, etc.

In [9]:
vals = np.array(range(24)).reshape((2, 3, 4))
a = o3c.Tensor(vals)
print("a.sum = {}\n".format(a.sum()))
print("a.min = {}\n".format(a.min()))
print("a.ArgMax = {}\n".format(a.argmax()))

a.sum = 276
Tensor[shape={}, stride={}, Int64, CPU:0, 0x56235015d670]

a.min = 0
Tensor[shape={}, stride={}, Int64, CPU:0, 0x562350162db0]

a.ArgMax = 23
Tensor[shape={}, stride={}, Int64, CPU:0, 0x562350107a50]



We can also specify the dimension to perform reduction operations.

In [10]:
# With specified dimension.
vals = np.array(range(24)).reshape((2, 3, 4))
a = o3c.Tensor(vals)

print("Along dim=0\n{}".format(a.sum(dim=(0))))
print("Along dim=(0, 2)\n{}\n".format(a.sum(dim=(0, 2))))

# Retention of reduced dimension.
print("Shape without retention : {}".format(a.sum(dim=(0, 2)).shape))
print("Shape with retention : {}".format(a.sum(dim=(0, 2), keepdim=True).shape))

Along dim=0
[[12 14 16 18],
 [20 22 24 26],
 [28 30 32 34]]
Tensor[shape={3, 4}, stride={4, 1}, Int64, CPU:0, 0x56234fb4f400]
Along dim=(0, 2)
[60 92 124]
Tensor[shape={3}, stride={1}, Int64, CPU:0, 0x5623503d3e50]

Shape without retention : SizeVector[3]
Shape with retention : SizeVector[1, 3, 1]


## 9. Slicing
We can using the below code for slicing. It is similar with pytorch and numpy.

In [11]:
vals = np.array(range(24)).reshape((2, 3, 4))
a = o3c.Tensor(vals)
print("a = \n{}\n".format(a))

# Indexing __getitem__.
print("a[1, 2] = {}\n".format(a[1, 2]))

# Slicing __getitem__.
print("a[1:] = \n{}\n".format(a[1:]))

# slice object.
print("a[:, 0:3:2, :] = \n{}\n".format(a[:, 0:3:2, :]))

# Combined __getitem__
print("a[:-1, 0:3:2, 2] = \n{}\n".format(a[:-1, 0:3:2, 2]))

a = 
[[[0 1 2 3],
  [4 5 6 7],
  [8 9 10 11]],
 [[12 13 14 15],
  [16 17 18 19],
  [20 21 22 23]]]
Tensor[shape={2, 3, 4}, stride={12, 4, 1}, Int64, CPU:0, 0x562268568d60]

a[1, 2] = [20 21 22 23]
Tensor[shape={4}, stride={1}, Int64, CPU:0, 0x562268568e00]

a[1:] = 
[[[12 13 14 15],
  [16 17 18 19],
  [20 21 22 23]]]
Tensor[shape={1, 3, 4}, stride={12, 4, 1}, Int64, CPU:0, 0x562268568dc0]

a[:, 0:3:2, :] = 
[[[0 1 2 3],
  [8 9 10 11]],
 [[12 13 14 15],
  [20 21 22 23]]]
Tensor[shape={2, 2, 4}, stride={12, 8, 1}, Int64, CPU:0, 0x562268568d60]

a[:-1, 0:3:2, 2] = 
[[2 10]]
Tensor[shape={1, 2}, stride={12, 8}, Int64, CPU:0, 0x562268568d70]



## 10. Logical operations
It supports various logical operations

In [12]:
a = o3c.Tensor(np.array([True, False, True, False]))
b = o3c.Tensor(np.array([True, True, False, False]))

print("a AND b = {}".format(a.logical_and(b)))
print("a OR b = {}".format(a.logical_or(b)))
print("a XOR b = {}".format(a.logical_xor(b)))
print("NOT a = {}\n".format(a.logical_not()))

# Only works for boolean tensors.
print("a.any = {}".format(a.any()))
print("a.all = {}\n".format(a.all()))

# If tensor is not boolean, 0 will be treated as False, while non-zero as true.
# The tensor will be filled with 0 or 1 casted to tensor's dtype.
c = o3c.Tensor(np.array([2.0, 0.0, 3.5, 0.0]))
d = o3c.Tensor(np.array([0.0, 3.0, 1.5, 0.0]))
print("c AND d = {}".format(c.logical_and(d)))

a AND b = [True False False False]
Tensor[shape={4}, stride={1}, Bool, CPU:0, 0x5623502f9ec0]
a OR b = [True True True False]
Tensor[shape={4}, stride={1}, Bool, CPU:0, 0x5622cc5efb10]
a XOR b = [False True True False]
Tensor[shape={4}, stride={1}, Bool, CPU:0, 0x5623502f9ec0]
NOT a = [False True False True]
Tensor[shape={4}, stride={1}, Bool, CPU:0, 0x5623502878b0]

a.any = True
Tensor[shape={}, stride={}, Bool, CPU:0, 0x562350157f30]
a.all = False
Tensor[shape={}, stride={}, Bool, CPU:0, 0x56229e6f7030]

c AND d = [False False True False]
Tensor[shape={4}, stride={1}, Bool, CPU:0, 0x5623516f0080]


## 11. Comparision operations
It supports various comparison operations.

In [13]:
a = o3c.Tensor([0, 1, -1])
b = o3c.Tensor([0, 0, 0])

print("a > b = {}".format(a > b))
print("a >= b = {}".format(a >= b))
print("a < b = {}".format(a < b))
print("a <= b = {}".format(a <= b))
print("a == b = {}".format(a == b))
print("a != b = {}".format(a != b))

# Throws exception if device/dtype is not shape.
# If shape is not same, then tensors should be broadcast compatible.
print("a > b = {}".format(a > b[0]))

a > b = [False True False]
Tensor[shape={3}, stride={1}, Bool, CPU:0, 0x562350442810]
a >= b = [True True False]
Tensor[shape={3}, stride={1}, Bool, CPU:0, 0x5623502e36c0]
a < b = [False False True]
Tensor[shape={3}, stride={1}, Bool, CPU:0, 0x562350442810]
a <= b = [True False True]
Tensor[shape={3}, stride={1}, Bool, CPU:0, 0x5623502e36c0]
a == b = [True False False]
Tensor[shape={3}, stride={1}, Bool, CPU:0, 0x562350442810]
a != b = [False True True]
Tensor[shape={3}, stride={1}, Bool, CPU:0, 0x5623502e36c0]
a > b = [False True False]
Tensor[shape={3}, stride={1}, Bool, CPU:0, 0x5623503da310]


## 12. Nearest Neighbor Search
Finally, Open3D is shipped with an efficient implementation for various types of neighbor searches, including K-nearest neighbor search, radius-based search, and hybrid search.

All of the search methods are supported on both CPU and GPU.

### 1. Data preparation

In this tutorial, we use ArmadilloMesh to demonstrate neighbor search methods.

In [14]:
armadillo = o3d.data.ArmadilloMesh()

[Open3D INFO] Downloading https://github.com/isl-org/open3d_downloads/releases/download/20220201-data/ArmadilloMesh.ply
[Open3D INFO] Downloaded to /root/open3d_data/download/ArmadilloMesh/ArmadilloMesh.ply


We use the vertices of mesh and convert it into Open3D tensor

In [15]:
mesh = o3d.io.read_triangle_mesh(armadillo.path)
points = np.asarray(mesh.vertices)
points_tensor = o3c.Tensor(points)

### 2. Initialize NearestNeighborSearch module

We can initialize ```NearestNeighborSearch``` module using the below code. 
Once the module is constructed, we can use it for searching neighbors with various search mode.

In [16]:
index = o3d.core.nns.NearestNeighborSearch(points_tensor)

### 3. Knn Search
To perform k-nearest neighbor search, we first need to setup ```NearestNeighborSearch``` module with ```knn_index()``` function.
Once the module is set up, we can perform knn search using ```knn_search``` method. 
```query``` tensor and ```knn``` value should be passed as input arguments.

In [17]:
index.knn_index()
indices, distances = index.knn_search(points_tensor, knn=10)

### 4. Radius Search
To perform radius-based neighbor search, we first need to setup ```NearestNeighborSearch``` module with ```fixed_radius_index()``` function.
Once the module is set up, we can perform knn search using ```fixed_radius_search``` method. 
```query``` tensor and ```radius``` value should be passed as input arguments.

In [18]:
index.fixed_radius_index(radius=0.1)
indices, distances, neighbor_splits = index.fixed_radius_search(
    points_tensor, radius=0.1
)

### 5. Hybrid Search
Fianlly, to perform hybrid search (radius search + knn search), we first need to setup ```NearestNeighborSearch``` module with ```hybrid_index()``` function.
Once the module is set up, we can perform knn search using ```hybrid_search``` method. 
```query``` tensor, ```radius``` value, and ```max_knn``` value should be passed as input arguments.

In [19]:
index.hybrid_index(radius=0.1)
indices, distances, num_neighbors = index.hybrid_search(
    points_tensor, radius=0.3, max_knn=10
)